In [ ]:
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
from scipy.spatial import distance
import random
from scipy.spatial import distance


In [ ]:
images = []
pca_features = []

In [ ]:
def init():
    model = keras.applications.VGG16(weights='imagenet', include_top=True)
    feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)

def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

def get_closest_images(query_image_idx, num_results=10):
    distances = [ distance.cosine(pca_features[query_image_idx], feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[1:num_results+1]
    return idx_closest

def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
    img = image.load_img(images[idx])
    img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
    thumbs.append(img)
    print(images[idx])
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image

def display_query(query_image, query_image_idx):
    plt.figure(figsize = (5,5))
    plt.imshow(query_image)
    plt.title("query image (%d)" % query_image_idx)
    
def display_result(results_image):
    plt.figure(figsize = (16,12))
    plt.imshow(results_image)
    plt.title("result images")
    
def upload(image_path):    
    # load image and extract features
    new_image, x = load_image(image_path)
    new_features = feat_extractor.predict(x)

    # project it into pca space
    new_pca_features = pca.transform(new_features)[0]
    return new_pca_features

def retrive_similar_images(new_pca_features):
    # calculate its distance to all the other images pca feature vectors
    distances = [ distance.cosine(new_pca_features, feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[0:5]  # grab first 5
    results_image = get_concatenated_images(idx_closest, 200)
    return results_image

